In [1]:
# pakettien lataus
!pip install gensim
!pip install nltk
!pip install pyLDAvis
import gensim
from gensim import corpora
from pprint import pprint
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import pyLDAvis.gensim_models
import string
import re
nltk.download('punkt')
nltk.download('snowball_data')
from nltk.stem.snowball import SnowballStemmer
import csv

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\annak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\annak\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


In [2]:
# aineiston lataus
file_path = "D:\\kokoaineisto.csv" # huom. tarkista sijainti
data = pd.read_csv(file_path, sep=';', dtype={'puhe': str})
data = data.iloc[:, :5]
data = data.dropna(subset=['puhe'])

In [3]:
# lisäsarake aineistoon tulosten lukua helpottamaan
data['new_column'] = data['puhe']

In [8]:
# poista välimerkit ja numerot
def remove_punctuation_and_numbers(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text
data['puhe'] = data['puhe'].apply(remove_punctuation_and_numbers).str.lower()

In [9]:
# stemmaus
stemmer = SnowballStemmer("finnish")
def stem_sentence(sentence):
    words = sentence.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)
data['puhe'] = data['puhe'].apply(stem_sentence)

In [11]:
# välitarkistus
data.head()

,id,puhuja,sukupuoli,puolue,puhe,new_column
0,1,"Ahvenjärvi, Sauli",m,Suomen Kristillisdemokraatit,arvois puhemies tänä käsiteltäv olev kansalais...,Arvoisa puhemies! Tänään käsiteltävänä olevall...
1,2,"Ahvenjärvi, Sauli",m,Suomen Kristillisdemokraatit,arvois puhemies minä aiva alu totea et tääl kä...,"Arvoisa puhemies! Minäkin aivan aluksi totean,..."
2,3,"Ahvenjärvi, Sauli",m,Suomen Kristillisdemokraatit,arvois puhemies valtioneuvosto ihmisoikeusselo...,Arvoisa puhemies! Valtioneuvoston ihmisoi-keus...
3,4,"Alanko-Kahiluoto, Outi",f,Vihreä liitto,arvois puhemies kyselytutkimust perust tiedä e...,Arvoisa puhemies! Kyselytutkimusten perusteell...
4,5,"Alanko-Kahiluoto, Outi",f,Vihreä liitto,arvois puhemies edustaj rajamäk tote juuri et ...,Arvoisa puhemies! Edustaja Rajamäki totesi juu...


In [13]:
# tokenointi
documents = data['puhe'].tolist()
tokenized_documents = [doc.lower().split() if isinstance(doc, str) else [] for doc in documents]

In [15]:
# stop-sanojen lataus
stopword_file_path = "D:\\finnish.txt" # huom. tarkista sijainti
with open(stopword_file_path, "r", encoding="utf-8") as file:
    finnish_stopwords = set(file.read().splitlines())

In [16]:
# stop-sanojen stemmaus
def stem_stopwords(stopwords):
    stemmed_stopwords = [stemmer.stem(word) for word in stopwords]
    return stemmed_stopwords
finnish_stopwords = stem_stopwords(finnish_stopwords)

In [17]:
# datan käsittelyä
filtered_documents = [[word for word in doc if word.lower() not in finnish_stopwords] for doc in tokenized_documents]
filtered_documents_strings = [' '.join(doc) for doc in filtered_documents]
data['puhe'] = filtered_documents_strings
dictionary = corpora.Dictionary(filtered_documents)
corpus = [dictionary.doc2bow(doc) for doc in filtered_documents]

In [18]:
# LDA-mallin koulutus
lda_model = gensim.models.LdaModel(corpus, num_topics=90, id2word=dictionary, passes=15) #huom. tarkista topicit ja iteraatiot

In [22]:
# tulosten visualisointi pyLDAvis-paketilla Jupyter Labissa
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

C:\Users\annak\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\annak\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\annak\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
C:\Users\annak\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
C:\Users\annak\AppData\Local\Programs\Python\Python312\Lib\site-pac

In [ ]:
# vaihtoehtoinen visualisointi Pythonissa
for topic_id, topic_words in lda_model.print_topics():
    print(f"Topic ID: {topic_id}")
    print(f"Words: {topic_words}")
    print()

In [26]:
# Yksittäisen topicin tulostaminen omaksi tiedostokseen
topic_index = 1 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic2.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)

In [27]:
topic_index = 5 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic6.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)

In [28]:
topic_index = 6 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic7.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)

In [29]:
topic_index = 7 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic8.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)

In [30]:
topic_index = 8 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic9.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)

In [31]:
topic_index = 0 # Huom! pyLDAvis topic numero -1
lda_output = lda_model.get_document_topics(corpus, per_word_topics=True)
topic_distribution = [dict(doc_topics) for doc_topics, _, _ in lda_output]
threshold = 0.3
relevant_docs_indices = [i for i, dist in enumerate(topic_distribution) if dist.get(topic_index, 0) > threshold]
relevant_docs_indices_sorted = sorted(relevant_docs_indices, key=lambda i: topic_distribution[i].get(topic_index, 0), reverse=True)
rows = []
for doc_index in relevant_docs_indices_sorted:
    doc_id = data.iloc[doc_index]['id']
    puhuja = data.iloc[doc_index]['puhuja']
    sukupuoli = data.iloc[doc_index]['sukupuoli']
    puolue = data.iloc[doc_index]['puolue']
    puhe = data.iloc[doc_index]['puhe']
    stemmaamaton_puhe = data.iloc[doc_index]['new_column']
    
    row = [doc_id, puhuja, sukupuoli, puolue, puhe, stemmaamaton_puhe]
    rows.append(row)
output_file = 'D:\\90topics-topic1.csv'  # huom. nimeä tiedosto ja tarkista sijainti
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Id', 'Puhuja', 'Sukupuoli', 'Puolue', 'Stemmattu puhe', 'Stemmaamaton puhe'])
    writer.writerows(rows)